# Microsoft Movie Studios Recommendations

![Microsoft Movie Studios](data/logo.png)

<h2>Overview

In this project we will help guide the executives at the newly founded Microsoft Movie Studios. We will analyze the <a href="www.imdb.com">IMDb</a>, <a href="https://www.rottentomatoes.com/">Rotten Tomatoes</a>, and <a href="https://www.boxofficemojo.com/">Box Office Mojo</a> datasets. We can use this information to make suggestions regarding the next steps regarding the creation of our first films, as well as potential acquisition decisions.

The first project our studio publishes will have major ramifications for our reputation both within the industry and without.
It's important for our future success to ensure our debut makes a major splash. 

<h2> Data Preparation

IMDb is the internet's movie largest database. We're interested in their data on Film Ratings, Films, and the Actors, Directors, Writers, and Producers involved in their creation.

In [14]:
# Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
%matplotlib inline
# Create connection and cursor objects to execute our SQLite queries
conn = sqlite3.connect("data/im.db")
cursor = conn.cursor()

In [15]:
q = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(q)
tables = cursor.fetchall()
table_names = [table[0] for table in tables]

In [16]:
# We now code loops through the table_names variable, and creates a variable with that name whose 
# value is the DataFrame associated with that table. It then appends these DataFrames to the list db
db = []
for t in table_names:
    vars().__setitem__(t, pd.read_sql(f"""
        SELECT *
        FROM {t}
        """, conn))
    db.append(vars()[t])

Let's first get an idea of the size, shape, columns, and datatypes within this database in order to inform further analysis

In [17]:
for x in range(len(db)):
    print(f"{table_names[x]}:")
    print(db[x].info())
    print()

movie_basics:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None

directors:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB
None

known_for:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638260 entries, 0 to 1638259
Data columns (

It's clear that much of this data will be unhelpful in our analysis, and that much if it is redundant. For example, directors, writers, and known_for tables contain information that is duplicated in the principals table.



For this reason, we'll be focusing on combining `primary_name`s from the `persons` table with their `category` and `movie_id` from the `principals` table.


Let's look a little closer at the movie_akas table.

As a new studio, it's  important for us to garner a positive reputation by creating our own original films. Therefore, we can eliminate analysis on data involving non-original films.

In [18]:
movie_akas_originals = movie_akas.loc[movie_akas['is_original_title'] == 1.0]
movie_akas_originals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44700 entries, 38 to 331700
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           44700 non-null  object 
 1   ordering           44700 non-null  int64  
 2   title              44700 non-null  object 
 3   region             6 non-null      object 
 4   language           4 non-null      object 
 5   types              44700 non-null  object 
 6   attributes         0 non-null      object 
 7   is_original_title  44700 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB


We see that nearly all the data within the `region`, `language`, and `attributes` columns are null. The `ordering`, `types`, and `is_original_title` columns are all redundant now as well. We drop these columns and name the resulting table `originals`

In [19]:
originals = movie_akas_originals.drop(labels=['region', 'language', 'attributes', 'is_original_title', 'types', 'ordering'], axis=1)

Now that we have the original movies, let's join them with the `movie_basics` and `movie_ratings` tables, in order to have the `title`, `start_year`, `runtime_minutes`, and `genres` columns from `movie_basics`, and `averagerating` and `numvotes` columns from `movie_ratings`.

In [20]:
originals = originals.set_index('movie_id')
movie_basics = movie_basics.set_index('movie_id')
movie_ratings = movie_ratings.set_index('movie_id')

In [21]:
originals = originals.join(movie_basics, how='inner', rsuffix='_mb')
originals = originals.join(movie_ratings, how='inner', rsuffix='_mb')
originals.drop(labels=['primary_title', 'original_title'], axis=1, inplace=True)
originals = originals.reset_index()

In [23]:
originals.head()

,movie_id,title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
1,tt0066787,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
4,tt0100275,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


Let's filter this table for a minimum of 500k `numvotes`, and sort by `averagerating`, as a sanity check.

In [29]:
originals.loc[(originals['numvotes'] >= 500000)].sort_values('averagerating', ascending=False).head()

,movie_id,title,start_year,runtime_minutes,genres,averagerating,numvotes
1556,tt1375666,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
206,tt0816692,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334
22558,tt4154756,Avengers: Infinity War,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926
4404,tt1675434,Intouchables,2011,112.0,"Biography,Comedy,Drama",8.5,677343
14124,tt2582802,Whiplash,2014,106.0,"Drama,Music",8.5,616916


Now let's look at the principals table

In [30]:
principals.head()

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [31]:
principals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   movie_id    1028186 non-null  object
 1   ordering    1028186 non-null  int64 
 2   person_id   1028186 non-null  object
 3   category    1028186 non-null  object
 4   job         177684 non-null   object
 5   characters  393360 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


We'll now limit the rows of this table to contain only those which pertain to an original movie, as denoted by the `movie_id` within our originals DataFrame. We can then join the resulting table with the `persons` and `movie_ratings` tables to obtain the names of the individuals, as well as the the ratings of their work.

In [32]:
originals_movie_ids = list(originals['movie_id'])
principals_bool = [movie_id in originals_movie_ids for movie_id in principals['movie_id']]
principals_in_orig = principals.loc[principals_bool]

In [33]:
principals_names = principals_in_orig.set_index('person_id').join(persons.set_index('person_id'), how='inner')
principals_names = principals_names.reset_index()
prin_names_ratings = principals_names.set_index('movie_id').join(movie_ratings, how='inner')
prin_names_ratings = prin_names_ratings.reset_index()

We'll now define a function to parse the `prin_names_ratings` by `category` column, to separate the roles of the individuals into actors, actresses, directors, writers, producers, and composers. The function will then average the rating and number of votes of their work, as well as create a new column for the sum of the number of votes on all of their work. We can use this information to draw conclusions about how best to cast roles for our first films

In [34]:
def create_top_tables(srole, source):
    """
    INPUT:
    This function is taking in a list of strings, and a cleaned principals table.
    The list of strings are values from the roles within the Persons Table, Category Column, which we are interested in. 
    The Principals table has already been cleaned to have non-original works and unncessary columns removed.
    
    OUTPUT:
    Outputs a DataFrame whose variable name is the relevant role, ordered by total number of votes.
    """
    vars()[srole] = source.loc[source['category'] == srole].groupby('person_id').mean().sort_values(by='averagerating', ascending=False)
    vars()[srole]['total_numvotes'] = source.loc[source['category'] == srole].groupby('person_id').sum()['numvotes']
    vars()[srole] = vars()[srole].drop(['birth_year', 'ordering', 'death_year'], axis=1).join(persons.set_index('person_id'), how='inner').drop(labels='death_year', axis=1).rename(columns={'numvotes': 'avg_numvotes'})
    vars()[srole] = vars()[srole].loc[vars()[srole]['avg_numvotes'] >= 200000].sort_values('total_numvotes', ascending=False)
    vars()[srole]['avg_numvotes'] = vars()[srole]['avg_numvotes'].astype('int64')
    return vars()[srole]

In [35]:
roles = ["actor", "actress", "director", "writer", "producer", "composer"]
for i in range(len(roles)):
    vars()[roles[i]] = create_top_tables(roles[i], prin_names_ratings)

Let's do a `.head()` on one of the resulting tables as a sanity check.

In [43]:
actor.head()

,averagerating,avg_numvotes,total_numvotes,primary_name,birth_year,primary_profession
person_id,,,,,,
nm0000375,7.530769,488930,6356093,Robert Downey Jr.,1965.0,"actor,producer,soundtrack"
nm0000138,8.088889,697068,6273617,Leonardo DiCaprio,1974.0,"actor,producer,writer"
nm0262635,7.266667,367209,5508138,Chris Evans,1981.0,"actor,producer,director"
nm0362766,7.361538,380252,4943284,Tom Hardy,1977.0,"actor,producer,writer"
nm1165110,6.933333,323257,4848858,Chris Hemsworth,1983.0,"actor,soundtrack,producer"


 Rotten Tomatoes is a review-aggregation website. Box Office Mojo tracks box-office revenue in a systematic way. It was purchased by IMDb in 2008. Using these three resources we can 

# 1. Budgets data cleaning

In [1]:
budgets = pd.read_csv("data/tn.movie_budgets.csv")
budgets.head()
budgets.shape
budgets.info()

NameError: name 'pd' is not defined

In [ ]:
budgets['production_budget'] = budgets['production_budget'].str.replace("$", "").str.replace(",", "").astype(int)
budgets['domestic_gross'] = budgets['domestic_gross'].str.replace("$", "").str.replace(",", "").astype(int)
budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace("$", "").str.replace(",", "").astype('int64')
budgets['release_date'] = pd.to_datetime(budgets['release_date'])
budgets.head()
budgets.info()

# Comments on budgets data cleaning
The file tn.movie_budgets.csv is now cleaned, and the data types are all configured for further analysis. 

There could potentially be an issue in the future where we have an issue with the in32 and int64 datatypes of the last 3 columns. If that occurs we can modify the data types at that time.

# 2. Loading and cleaning bom.movie_gross

In [ ]:
Bom = pd.read_csv("Data/bom.movie_gross.csv")

In [ ]:
Bom.head()
Bom.info()
Bom.isna().sum()

# Comments on values for BOM Movie Gross

- 5 missing values in studio. Will remove, since they're only 5 out of 3387.
- 28 missing values in domestic gross, will turn them all to 0. All str, will have to convert them to float, once all null's have been replaced with 0. Inspected them to make sure they were just foreign films.
- 1350 missing values in foreign gross (turn to 0), will turn them all to 0. All str, will have to convert them to float, once all null's have been replaced with 0. Domestic films that didn't have foreign gross. One has a comma, so have to take it out to normalize data.

In [ ]:
Bom['foreign_gross'] = Bom['foreign_gross'].str.replace(',','')
Bom['foreign_gross'] = Bom['foreign_gross'].astype(float)
Bom['foreign_gross'] = Bom['foreign_gross'].fillna(0)
Bom['domestic_gross'] = Bom['domestic_gross'].fillna(0)
Bom = Bom.dropna(subset = ['studio'])

In [ ]:
# Will create a new column, total_gross, to be able to compare the aggregate gross down the line.
Bom['total_gross'] = Bom['domestic_gross'] + Bom['foreign_gross']

In [ ]:
#Upon trying to join (in an exploratory notebook) Bom dataframe with the budgets dataframe,
#we realized that we should change one of the 'domestic_gross' column name to be able to
#differentiate.
Bom.rename(columns={'domestic_gross':'domestic_gross_bom'},inplace=True)

In [ ]:
Bom_titles = list(Bom['title'])
"""
We realized that in order to be able to give any sort of business advice we'd need to see
what these movies cost to make. Even though the budgets table has domestic and worldwide
gross, it does not have the studio that makes them. 

We will do a left join on the budgets dataframe with the movie titles as the overlap
column. In order to see which movie titles are in both dataframes we are creating a new 
column in budgets that returns True if the movie title overlaps. This is why
the above variable was created.

This overlap will allow us to dig deeper into what the investment return metrics for a succesful
movie studio look like. 

"""
def compare_title(title):
    if title in Bom_titles:
        return True
    else:
        return False
    
"""
The above function takes in a string, in this case a movie title, and returns true if that
string is in the provided Bom movie title list.

"""

In [ ]:
budgets['Bom'] = budgets['movie'].map(compare_title)
budgets.head()

# 2.5 Left join bom to budgets

In [ ]:
Bom_budg_ljn_on_title = budgets.join(Bom.set_index(['title']), on=['movie'])
"""
Below we're taking a look at all the movies that overlap in bom and budgets, and have a non
null value in the studio column.
"""
Bom_budg_ljn_on_title.loc[(Bom_budg_ljn_on_title['Bom']== True) & (Bom_budg_ljn_on_title['studio'] != Bom_budg_ljn_on_title['studio'].isna())]

In [ ]:
"""
Creating a separate dataframe to analyze each studio's worldwide total gross in usd, 
total production budget in usd, lifetime return in usd (which is calculated via
'worldwide total gross' - 'total production budget'), lifetime return as a % (which is
calculated via (('worldwide total gross' / 'total production budget')-1)), and the total
number of films per studio that these return metrics are based off of. Which in some cases
may be a very small sample of the total catalogue that studio has.

"""
studio_gross = Bom_budg_ljn_on_title.groupby('studio')['worldwide_gross'].sum().astype('int64')
studio_gross = pd.DataFrame(studio_gross)
studio_gross['total_production_budget'] = Bom_budg_ljn_on_title.groupby('studio')['production_budget'].sum().astype('int64')
studio_gross['$_lifetime_return'] = studio_gross['worldwide_gross'] - studio_gross['total_production_budget']
studio_gross['%_lifetime_return'] = (studio_gross['worldwide_gross']/studio_gross['total_production_budget']) - 1
studio_gross['#_of_films'] = Bom_budg_ljn_on_title.groupby('studio')['Bom'].sum().astype('int64')
studio_gross.reset_index(inplace=True)
studio_gross.describe()

In [ ]:
"""
There appears to only be 1,246 movies that overlap. Want to see mean, median, and 75% IQR
for all values. Upon review, in order to give a recommendation on a studio we'll need to
narrow down our search to studios that have at least produced more than 10 (75% IQR) or
12 movies (mean). This will be our cutoff, so that we can't recommend a one hit wonder
studio or so to speak.
"""
studio_gross.loc[studio_gross['#_of_films'] > 10].sort_values('#_of_films',ascending=False)
studio_usd_sort = studio_gross.sort_values('$_lifetime_return',ascending=False)
studio_pct_sort = studio_gross.loc[studio_gross['#_of_films'] > 10].sort_values('%_lifetime_return',ascending=False)

In [ ]:
"""
1st plot is a bar chart showing in descending order the top 20 studios based on lifetime
return in usd.

2nd plot is a bar chart showing in descending order the top 20 studios based on lifetime
return as a %.

3rd chart for exploratory purposes could be a scatter plot with number of movies and returns

"""
fig, ax = plt.subplots(figsize=(15,6))
ax.bar(x=studio_usd_sort['studio'].head(20),height=studio_usd_sort['$_lifetime_return'].head(20))

fig, ax = plt.subplots(figsize=(15,6))
ax.bar(x=studio_pct_sort['studio'].head(20),height=studio_pct_sort['%_lifetime_return'].head(20));
;

# 3. Loading in and cleaning RT movie info

In [ ]:
Rt = pd.read_csv('Data/rt.movie_info.tsv',sep = '\t')
Rt.head()
Rt.info()
Rt.isna().sum()

# Missing data comments
- synopsis, only missing 62 entries
- rating missing 3 entries
- genre missing 8 values, since our analysis in this section will be based upon genres we'll remove the nulls here. Removing them because they're a very small portion of the dataset.
- director, missing 199 entries
- writer, missing 449 entries
- theater_date, missing 359 entries
- dvd_date, missing 359 entries as well
- currency, missing 1210 entries
- box-office, missing 1210 entries as well.
- runtime, missing 20 entries
- studio, missing 1066 entries
- I don't have any move titles....

# Cleaning the RT dataframe
- Should condense genre, to a category | subcategory
- Should turn the dates to date value
- Should turn the box office number into an int or float
- Should standardize runtime into mins
- Will remove entries without synopsis as we won't be able to source movie title, and double checked that they do not have box office numbers either.

In [ ]:
# Cleaning up the box office numbers from str to int
Rt['box_office'] = Rt['box_office'].str.replace(',','')

# Cleaning up the theater and dvd dates to datetime format.
Rt['theater_date'] = pd.to_datetime(Rt['theater_date'])
Rt['dvd_date'] = pd.to_datetime(Rt['dvd_date'])

# Findings on data exploration
Was going to remove the null synopsis entries, and try to match the movie titles in the
budgets CSV titles to the movie title in the synopsis string. Every synopsis mentions the title of the movie they're talking about at least once. The problem with trying to return the movie title string if it's in the synopsis is that for a single word or number movie title, it would return the wrong movie title. Example being 'her', the word her appears a myriad of times through a multitude of synopsis, which would lead to incorrect movie titles being assigned.

Rt = Rt.dropna(subset=['synopsis'])

In [ ]:
"""
Decided to pivot from trying to join this dataframe with budgets, and now we are going to 
standardize the 'genre' format in order to produce an analysis based on 'genre' and 'fresh'
rating. Going to limit the 'genre' column to just the first two that currently show in 'genre'.
This will lower the unique genre's from 299 to 78.

We want to find out if there's a skew for certain genres, although more popular movies
in any genre will always skew a genre due to the number of ratings.
"""
def genre_condenser(genre):
    new_genre = genre.split(sep='|')
    try:
        if 'Classics' in new_genre:
            new_genre.remove('Classics')
            new_genre = f'{new_genre[0]}|{new_genre[1]}'
            return new_genre
        else:
            new_genre = f'{new_genre[0]}|{new_genre[1]}'
            return new_genre
    except:
        new_genre = f'{new_genre[0]}'
        return new_genre

#Dropping the Nan values associated with genre.
Rt = Rt.dropna(subset = ['genre'])
Rt['genre'] = Rt['genre'].map(genre_condenser)

# 2.5 Loading in the RT Reviews
- When at least 60% of reviews for a movie or TV show are positive, a red tomato is displayed to indicate its Fresh status.

- When less than 60% of reviews for a movie or TV show are positive, a green splat is displayed to indicate its Rotten status.

https://www.rottentomatoes.com/about

In [ ]:
Rt_Reviews = pd.read_csv('Data/rt.reviews.tsv',delimiter='\t',encoding='cp850')
Rt_Reviews.info()
Rt_Reviews.isna().sum()

# Missing values
- review         5563
- rating        13517
- critic         2722
- publisher       309
- No missing values for 'fresh', which let's us fully exlpore our 'fresh' to 'genre'
    analysis.

In [ ]:
#It appears that there are only 1135 unique movies according to the unique id count.
#So we have 425 less unique movies to compare. Total movies in the RT movie info is 1560.
Rt_Reviews['id'].value_counts()

In [ ]:
# In order to join, need to change the rating label col name for one of them.
# Upon joining it looks like we have five less genres. 78 to 73.
Rt_Reviews.rename(columns={'rating':'mov_rating'}, inplace=True)
Rt_id_left_join = Rt_Reviews.join(Rt.set_index(['id']), on=['id'])

# 4. Loading in and cleaning tmdb.movies

In [ ]:
tmdb = pd.read_csv('Data/tmdb.movies.csv')

In [ ]:
#Dropping Unnamed: 0
#No missing values
tmdb.drop(columns=['Unnamed: 0'],inplace=True)
tmdb.info()

In [ ]:
#Getting rid of the empty genre_id entries, around 2000 entries eliminated from this. Have
#around 1000 genres, will deal with this below by creating a fucntion to map out the dataframe.
tmdb = tmdb.loc[tmdb['genre_ids'] != '[]']
tmdb['genre_ids'].value_counts()

In [ ]:
#TMDB genre ID tags https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee
#Created a dictionary, based on the id values found on the website!
tmdb_genres= {"28":"Action",
            "12":"Adventure",
            "16":"Animation",
            "35":"Comedy",
            "80":"Crime",
            "99":"Documentary",
            "18":"Drama",
            "10751":"Family",
            "14":"Fantasy",
            "36":"History",
            "27":"Horror",
            "10402":"Music",
            "9648":"Mystery",
            "10749":"Romance",
            "878":"Science Fiction",
            "10770":"TV Movie",
            "53":"Thriller",
            "10752":"War",
            "37":"Western"}

In [ ]:
"""
Have around 1000 types of genres, in order to give a recommendation based on genres, will
need to cap the genre's per id. Tried capping it to three, but still gave us 595 unique genres.
Decided to cap it to two, which brings it down to 164 which is paletable for an analysis.

The function below takes in the str, which is a list of genre ids, and converts it to a
list of genre ids . Then once we have the list of ids it checks each id through the tmdb genre list
and adds the genre name to a blank list. If the genre name is already there then it will not
add it to the list again. The function then checks if the list of genre names is at two, and
if so it returns the string version of the genre name list.

Tested to make sure that genre names weren't coming out as double unique identifiers, ie
'Drama'|'Comedy' and 'Comedy'|'Drama'.

"""

def genre_id_conv(genre_list):
    genre_list = genre_list.replace('[','').replace(']','').split(sep=',')
    genre_name = []
    for key,val in tmdb_genres.items():
        for genre_id in genre_list:
            if key in genre_id:
                if val not in genre_name and len(genre_name) < 2:
                    genre_name.append(val)
    return str(genre_name)

tmdb['genre_names'] = tmdb['genre_ids'].map(genre_id_conv)

In [ ]:
tmdb.describe()

<h2> Feature Engineering

<h2>Data Analysis

<h2> Conclusions